# [Learn Quantum Computing with Python and Q#](https://www.manning.com/books/learn-quantum-computing-with-python-and-q-sharp?a_aid=learn-qc-granade&a_bid=ee23f338)<br>Chapter 10 Exercise Solutions
----
> Copyright (c) Sarah Kaiser and Chris Granade.
> Code sample from the book "Learn Quantum Computing with Python and Q#" by
> Sarah Kaiser and Chris Granade, published by Manning Publications Co.
> Book ISBN 9781617296130.
> Code licensed under the MIT License.


### Exercise 10.1 

**Use `DumpMachine` to see how `Controlled Z` acts on the uniform superposition state
$\left| + + + \cdots +\right\rangle$ .**



In [1]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arrays;

operation ReportCZEffect(nQubits : Int) :  Unit {
    using (register = Qubit[nQubits]) {
        // Prepares the register in the |+++ ...+〉state.
        ApplyToEach(H, register);

        Message("Before the Controlled Z operation:");
        DumpRegister((), register);

        Controlled Z(Most(register), Tail(register));

        Message("After the Controlled Z operation:");
        DumpRegister((), register);
        
        ResetAll(register);
    }
} 

ReportCZEffect

In [2]:
%simulate ReportCZEffect nQubits=3

Before the Controlled Z operation:


|0⟩	0.35355339059327384 + 0𝑖
|1⟩	0.35355339059327384 + 0𝑖
|2⟩	0.35355339059327384 + 0𝑖
|3⟩	0.35355339059327384 + 0𝑖
|4⟩	0.35355339059327384 + 0𝑖
|5⟩	0.35355339059327384 + 0𝑖
|6⟩	0.35355339059327384 + 0𝑖
|7⟩	0.35355339059327384 + 0𝑖

After the Controlled Z operation:


|0⟩	0.35355339059327384 + 0𝑖
|1⟩	0.35355339059327384 + 0𝑖
|2⟩	0.35355339059327384 + 0𝑖
|3⟩	0.35355339059327384 + 0𝑖
|4⟩	0.35355339059327384 + 0𝑖
|5⟩	0.35355339059327384 + 0𝑖
|6⟩	0.35355339059327384 + 0𝑖
|7⟩	-0.35355339059327384 + 0𝑖

()

You can see that applying the `Controlled Z` operation has flipped the phase of the all-ones computational basis state. In particular, the $|7\rangle = |111\rangle$ basis state has been mapped to $-|7\rangle$ while all other basis states are left alone.
We say that the `Controlled Z` operation is a _reflection about the $|111\rangle$ state_.

----
### Exercise 10.2

**The operation `ReflectAboutMarkedState` uses `X(flag); H(flag);` to prepare the flag qubit, however you could also have used `H(flag); Z(flag);`.**

**Using either or both of QuTiP and `AssertOperationsEqualReferenced`, prove that these two ways of preparing your flag qubit give you the same reflection.**

In [3]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;
open GroverSearch;

operation UseXHPreparation(
    markedItemOracle : ((Qubit[], Qubit) => Unit is Adj),
    inputQubits : Qubit[]
)
: Unit is Adj {
    using (flag = Qubit()) {
        within {
            X(flag);
            H(flag);
        } apply{
            markedItemOracle(inputQubits, flag);                                       
        }
    }
}

operation UseHZPreparation(
    markedItemOracle : ((Qubit[], Qubit) => Unit is Adj),
    inputQubits : Qubit[]
)
: Unit is Adj {
    using (flag = Qubit()) {
        within {
            H(flag);
            Z(flag);
        } apply{
            markedItemOracle(inputQubits, flag);                                       
        }
    }
}

operation ProveEqualPreparations(nItems : Int, idxMarkedItem : Int): Unit {
    let markItem = ApplyOracle(idxMarkedItem, _, _);
    AssertOperationsEqualReferenced(BitSizeI(nItems-1),
        UseXHPreparation(markItem, _),
        UseHZPreparation(markItem, _)
    );
}

UseXHPreparation, UseHZPreparation, ProveEqualPreparations

In [4]:
%simulate ProveEqualPreparations nItems=8 idxMarkedItem=2

()

----
### Exercise 10.3

**Try writing out what `(ControlledOnInt(4, X))(register, flag)` does to the state of `register + [flag]`, using either Dirac notation (check out Chapters 2 and 4 if you need a refresher) or by writing down a unitary matrix that can be used to simulate `(ControlledOnInt(4, X))` acting on a three-qubit register and a flag qubit.
Try doing the same, but for `(ControlledOnInt(4, Z))`.**

> **HINT**: since `(ControlledOnInt(4, X))` acts on four qubits in this example (three control qubits and a target qubit), your unitary matrix should be a $16 \times 16$ matrix.

Using Dirac notation for brevity, `(ControlledOnInt(4, X))` can be represented by the unitary matrix
$$
    U_{CX_4} = \left(
        |000\rangle\langle000| +
        |010\rangle\langle010| +
        |011\rangle\langle011| +
        |100\rangle\langle100| +
        |101\rangle\langle101| +
        |110\rangle\langle110| +
        |111\rangle\langle111|
    \right) \otimes 𝟙 +
    |001\rangle\langle001| \otimes X.
$$

Put differently, the `(ControlledOnInt(4, X))` operation leaves all computational basis states alone except those for which the control register is in the $|001\rangle = |4\rangle$ state (recall, Q# uses little-endian notation, so that $4$ is written out as $001$).

----
### Exercise 10.4

**Try changing the definition of the oracle to control on a different integer.
Does this change the output when you run Grover's algorithm?**

In [5]:
%simulate RunGroverSearch nItems=8 idxMarkedItem=2

marked 2 and found 2.


()

Chaning the integer it is searching for (also controling on) does not change the output.

----
### Exercise 10.5

**Try modifying your implementation of Grover's algorithm to apply twice as many iterations, and use `DumpMachine` to look at the state that results.
Does that seem like what you'd expect from applying a rotation twice?**

In [6]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Math;
open GroverSearch;

operation SearchForMarkedItemLonger(
    nItems : Int,
    markItem : ((Qubit[], Qubit) => Unit is Adj)
)
: Int {
    using (qubits = Qubit[BitSizeI(nItems - 1)]) {
        PrepareInitialState(qubits);
        
        for (idxIteration in 0..NIterations(BitSizeI(nItems)) - 1) {
            ReflectAboutMarkedState(markItem, qubits);
            ReflectAboutInitialState(PrepareInitialState, qubits);
        }
        Message("After the normal number of iterations:");        
        DumpMachine();

        for (idxIteration in 0..NIterations(BitSizeI(nItems)) - 1) {
            ReflectAboutMarkedState(markItem, qubits);
            ReflectAboutInitialState(PrepareInitialState, qubits);
        }
        Message("After twice the normal number of iterations:");
        DumpMachine();

        return MeasureInteger(LittleEndian(qubits));
    }
}

operation RunGroverSearchLonger(nItems : Int, idxMarkedItem : Int) : Unit {
    let markItem = ApplyOracle(idxMarkedItem, _, _);
    let foundItem = SearchForMarkedItemLonger(nItems, markItem);
    Message($"Marked {idxMarkedItem} and found {foundItem}.");
}

SearchForMarkedItemLonger, RunGroverSearchLonger

In [7]:
%simulate RunGroverSearchLonger nItems=8 idxMarkedItem=4

After the normal number of iterations:


|0⟩	0.3093592167691151 + 0𝑖
|1⟩	0.30935921676911504 + 0𝑖
|2⟩	0.3093592167691151 + 0𝑖
|3⟩	0.3093592167691151 + 0𝑖
|4⟩	-0.5745242597140706 + 0𝑖
|5⟩	0.30935921676911504 + 0𝑖
|6⟩	0.30935921676911504 + 0𝑖
|7⟩	0.30935921676911504 + 0𝑖

After twice the normal number of iterations:


|0⟩	-0.00552427172801985 + 0𝑖
|1⟩	-0.00552427172801985 + 0𝑖
|2⟩	-0.0055242717280197945 + 0𝑖
|3⟩	-0.0055242717280199055 + 0𝑖
|4⟩	-0.9998931827716052 + 0𝑖
|5⟩	-0.005524271728019961 + 0𝑖
|6⟩	-0.005524271728019919 + 0𝑖
|7⟩	-0.005524271728019877 + 0𝑖

Marked 4 and found 4.


()

What you can see here is that if your continue reflecting the state past the ideal number of reflections you get that at twice the optimal number of reflections the marked state is the least likely to be measured (it has the smallest amplitude). This makes some sense as there is a lot of trigonometry involved in finding the optimum number of reflections such that you might see periodic features like this.

----
### Exercise 10.6

**Use `AssertOperationsEqualReferenced` to prove that applying the `T` operation four times does the same thing as applying `Z` once.**

In [8]:
open Microsoft.Quantum.Diagnostics;

operation ProveFourTEqualsOneZ() : Unit {
    AssertOperationsEqualReferenced(1,
        ApplyToEachA(OperationPowA(T, 4), _),
        ApplyToEachA(Z, _)
    );
}

ProveFourTEqualsOneZ

In [9]:
%simulate ProveFourTEqualsOneZ

()

**There's another operation `S` that can be thought of as the square root of `Z` (a 90° rotation about the s $Z$-axis); check that applying `T` twice is the same as applying `S` once.**

In [10]:
open Microsoft.Quantum.Diagnostics;

operation ProveTwoTEqualsOneS() : Unit {
    AssertOperationsEqualReferenced(1,
        ApplyToEachA(OperationPowA(T, 2), _),
        ApplyToEachA(S, _)
    );
}

ProveTwoTEqualsOneS

In [11]:
%simulate ProveTwoTEqualsOneS

()

----
### Exercise 10.7

**Why don't we need to reset the register of qubits allocated in `EstimateCcnotResources` as shown in Figure 10.10?**

You don't need to reset the register because the `CCNOT` operation didn't do anything. Both control qubits were in the zero state, so that means the operation should do nothing.
Put differently, the all-zeros state is an eigenstate of the `CCNOT` operation.

----
### Exercise 10.8

**How does the number of `T` operation calls change as you increase the number of control qubits? A rough trend is fine.**

> **HINT:** As you saw above, a controlled-NOT operation with an arbitrary number of qubits can be written as `Controlled X(Most(qs), Tail(qs));`, using functions provided by the `Microsoft.Quantum.Arrays` namespace.

In [12]:
open Microsoft.Quantum.Arrays;

operation EstimateCNotScaling(nQubits : Int) : Unit{
    using (register = Qubit[nQubits]) {
        Controlled X(Most(register), Tail(register));
    }
}


EstimateCNotScaling

In [13]:
%estimate EstimateCNotScaling nQubits=4

Metric,Sum,Max
CNOT,30,30
QubitClifford,6,6
R,0,0
Measure,0,0
T,21,21
Depth,14,14
Width,5,5
BorrowedWidth,0,0


In [14]:
%estimate EstimateCNotScaling nQubits=5

Metric,Sum,Max
CNOT,50,50
QubitClifford,10,10
R,0,0
Measure,0,0
T,35,35
Depth,23,23
Width,7,7
BorrowedWidth,0,0


In [15]:
%estimate EstimateCNotScaling nQubits=6

Metric,Sum,Max
CNOT,70,70
QubitClifford,14,14
R,0,0
Measure,0,0
T,49,49
Depth,32,32
Width,9,9
BorrowedWidth,0,0


In [16]:
%estimate EstimateCNotScaling nQubits=7

Metric,Sum,Max
CNOT,90,90
QubitClifford,18,18
R,0,0
Measure,0,0
T,63,63
Depth,41,41
Width,11,11
BorrowedWidth,0,0


So the data we see for this correlation is as follows as pairs (# of qubits, # of `T` operations):

| # of control qubits | # of `T` operation calls needed |
|---|---|
| 3 | 7 |
| 4  | 21 |
| 5 | 35 |
| 6 | 49 |
| 7 | 63 |

What you can see here is that the more control qubits you have on an operation, the more `T` operations are needed to implement it.

----
### Epilogue

_The following cell logs what version of the components this was last tested with._

In [17]:
%version

Component,Version
iqsharp,0.12.20070124
Jupyter Core,1.4.0.0
.NET Runtime,".NETCoreApp,Version=v3.1"
